# Scale bar detection using an image processing algorithm

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from utils import detect_black_scale_bar

In [2]:
images_folder = "C:/Users/elyes/Documents/UNIL ants/EPFL/original/"

In [3]:
from utils import resize_image
print(resize_image(images_folder + "anic32-900190-1_p_1.jpg", img_name="anic32-900190-1_p_1.jpg"))

./resized_images\anic32-900190-1_p_1.jpg


In [5]:
from utils import process_images_from_csv
process_images_from_csv("thorax_only.csv", "annotations.csv", images_folder)

Scale bar not detected for image: ./resized_images\anic32-900190-1_p_1.jpg
Scale bar not detected for image: ./resized_images\antweb1008066_p_1.jpg
Scale bar not detected for image: ./resized_images\antweb1038001_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0005455_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0005691_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0005757_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0059749_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0062173_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0066723_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0102301_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0102549_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0102572_p_1.jpg
Scale bar not detected for image: ./resized_images\casent0103422_p_1.jpg
Scale bar not detected for image: ./resized_image

KeyboardInterrupt: 

In [9]:
from utils import draw_points_and_scale
import pandas as pd
import numpy as np

# Define the coordinates (example points)
df = pd.read_csv("annotations.csv").dropna()
row = df.iloc[np.random.randint(0, 1000)]
x1, y1 = row["x1_lmk"], row["y1_lmk"]
x2, y2 = row["x2_lmk"], row["y2_lmk"]
x3, y3 = row["x1_bar"], row["y1_bar"]
x4, y4 = row["x2_bar"], row["y2_bar"]

# Scale bar positions ((start_x, start_y), (end_x, end_y))
scale_bar_points = ((int(x3), int(y3)), (int(x4), int(y4)))  # Detected scale bar coordinates

# Thorax positions ((start_x, start_y), (end_x, end_y)) - Approximate example
thorax_points = ((int(x1), int(y1)), (int(x2), int(y2)))

# the image is resized to match the 640x640 image dimensions
image_path = "./resized_images/" + row["ant"]  # Replace with your image file path

draw_points_and_scale(image_path, scale_bar_points, thorax_points)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
